In [6]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error
import json
import ast
import re
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV


In [7]:
# data_list = []

# # Path to the file
# file_path = 'goodreads_reviews_children.json.gz'

# with gzip.open(file_path, 'rt', encoding='utf-8') as file:
#     for line in file:
#         try:
#             # Safely evaluate each line as a dictionary
#             data_list.append(ast.literal_eval(line.strip()))
#         except Exception as e:
#             print(f"Error processing line: {line}\n{e}")

# # Display the first 5 entries to verify


In [8]:
data = []
file_path = 'renttherunway_final_data.json.gz'
with gzip.open(file_path, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            data.append(json.loads(line.strip()))
        except Exception as e:
            print(f"Error processing line: {line}\n{e}")

uir_data = [entry for entry in data
            if all(field in entry and entry[field] for field in ['rating', 'user_id', 'item_id'])]
svd_data = [
    (d['user_id'], d['item_id'], int(d['rating'])) 
    for d in uir_data if 'user_id' in d and 'item_id' in d and 'rating' in d
]


In [9]:
print(len(data))
data[0]

192544


{'fit': 'fit',
 'user_id': '420272',
 'bust size': '34d',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016'}

In [10]:
data = [
    entry for entry in data
    if all(field in entry and entry[field] for field in ['rating', 'review_text', 'review_summary', 'size', 'age', 'review_date', 'rented for'])
]


In [11]:
print(len(data))


190971


In [12]:
def MSE(y, ypred):
    diffs = [(a-b)**2 for (a,b) in zip(y,ypred)]
    return sum(diffs) / len(diffs)

In [13]:
def feat1(d):
    return [1, len(d['review_text'])] 

In [14]:

dataTrain = data[:int(len(data)*0.8)]
dataTest = data[int(len(data)*0.8):]

In [15]:
Xtrain = [feat1(d) for d in dataTrain]
ytrain = [int(d['rating']) for d in dataTrain]
Xtest = [feat1(d) for d in dataTest]
ytest = [int(d['rating']) for d in dataTest]

In [16]:
unique_ratings = set(entry['rating'] for entry in data if 'rating' in entry)
unique_rent = set(entry['rented for'] for entry in data if 'rented for' in entry)
print(unique_ratings, unique_rent)

{'8', '6', '2', '10', '4'} {'wedding', 'date', 'everyday', 'party', 'vacation', 'other', 'party: cocktail', 'formal affair', 'work'}


In [17]:
unique_ratings = set(entry['rating'] for entry in data if 'rating' in entry)
unique_ratings

{'10', '2', '4', '6', '8'}

In [ ]:
def feat1(d):
    return [1, len(d['review_text'])] 

In [ ]:
Xtrain = [feat1(d) for d in dataTrain]
ytrain = [int(d['rating']) for d in dataTrain]
Xtest = [feat1(d) for d in dataTest]
ytest = [int(d['rating']) for d in dataTest]

In [18]:
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse


2.0509410951628015

In [19]:
def feat2(d):
    return [1, len(d['review_text']), len(d['review_summary'])] 

In [20]:
Xtrain = [feat2(d) for d in dataTrain]
Xtest = [feat2(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse


2.043165781399794

In [21]:
def feat3(d):
    return [1, len(d['review_text']), len(d['review_summary']), d['review_text'].count('!') + d['review_summary'].count('!')] 

In [22]:
Xtrain = [feat3(d) for d in dataTrain]
Xtest = [feat3(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.9290804311924041

In [23]:
def feat4(d):
    return [1, d['review_text'].count('!') + d['review_summary'].count('!')] 

In [24]:
Xtrain = [feat4(d) for d in dataTrain]
Xtest = [feat4(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.948420335125565

In [25]:
def feat5(d):
    return [1, len(d['review_text']), 
            len(d['review_summary']), 
            d['review_text'].count('!') + d['review_summary'].count('!'),
            int(d['age']),
            ] 

In [26]:
Xtrain = [feat5(d) for d in dataTrain]
Xtest = [feat5(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.9287141659151328

In [27]:
def feat(d):
    return [1, len(d['review_text']), 
            len(d['review_summary']), 
            d['review_text'].count('!') + d['review_summary'].count('!'),
            int(d['age']),
            int(d['size'])
            ] 

In [28]:
Xtrain = [feat(d) for d in dataTrain]
Xtest = [feat(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.9252668614896955

In [29]:
def feat(d):
    return [1, len(d['review_text']), 
            len(d['review_summary']), 
            d['review_text'].count('!') + d['review_summary'].count('!'),
            int(d['age']),
            int(d['size']),
            int(d['review_date'][-4:])
            ] 
Xtrain = [feat(d) for d in dataTrain]
Xtest = [feat(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse


1.91424095555222

In [30]:
def feat(d):
    age = int(d['age'])
    size = int(d['size'])
    
    return [1, len(d['review_text']), 
            len(d['review_summary']), 
            d['review_text'].count('!') + d['review_summary'].count('!'),
            int(d['review_date'][-4:]),
            age,
            size
            ] 
Xtrain = [feat(d) for d in dataTrain]
Xtest = [feat(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.9142409555522135

In [31]:
def feat(d):
    age = int(d['age'])
    size = int(d['size'])
    age_size_interaction = age * size
    return [1, len(d['review_text']), 
            len(d['review_summary']), 
            d['review_text'].count('!') + d['review_summary'].count('!'),
            int(d['review_date'][-4:]),
            age_size_interaction
            ] 
Xtrain = [feat(d) for d in dataTrain]
Xtest = [feat(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.9148641067333887

In [32]:
def feat(d):
    age = int(d['age'])
    size = int(d['size'])
    age_size_interaction = age * size
    sentiment_words = ['love', 'hate', 'perfect', 'amazing', 'awful', 'good', 'bad']
    sentiment_count = sum(d['review_text'].lower().count(word) + d['review_summary'].lower().count(word) for word in sentiment_words)
    return [1, len(d['review_text']), 
            len(d['review_summary']), 
            d['review_text'].count('!') + d['review_summary'].count('!'),
            int(d['review_date'][-4:]),
            age_size_interaction,
            sentiment_count
            ] 
Xtrain = [feat(d) for d in dataTrain]
Xtest = [feat(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse


1.838722954905515

In [44]:
rented_for_categories = ['date', 'party', 'formal affair', 'everyday', 'other', 'wedding', 'vacation', 'party: cocktail', 'work']

def feat(d):
    age = int(d['age'])
    size = int(d['size'])
    age_size_interaction = age * size
    sentiment_words = ['love', 'hate', 'perfect', 'amazing', 'awful', 'good', 'bad']
    sentiment_count = sum(d['review_text'].lower().count(word) + d['review_summary'].lower().count(word) for word in sentiment_words)
    unique_words = len(set(re.findall(r'\w+', d['review_text'] + " " + d['review_summary'])))
    rented_for_encoded = [1 if d.get('rented for', '').lower() == category else 0 for category in rented_for_categories]
    return [1, len(d['review_text']), 
            len(d['review_summary']), 
            d['review_text'].count('!') + d['review_summary'].count('!'),
            int(d['review_date'][-4:]),
            age_size_interaction,
            sentiment_count,
            ] + rented_for_encoded
Xtrain = [feat(d) for d in dataTrain]
Xtest = [feat(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.8279770862346172

In [34]:

def feat(d):
    age = int(d['age'])
    size = int(d['size'])
    age_size_interaction = age * size
    sentiment_words = ['love', 'hate', 'perfect', 'amazing', 'awful', 'good', 'bad']
    sentiment_count = sum(d['review_text'].lower().count(word) + d['review_summary'].lower().count(word) for word in sentiment_words)
    unique_words = len(set(re.findall(r'\w+', d['review_text'] + " " + d['review_summary'])))
    contains_recommend = 1 if 'recommend' in d['review_text'].lower() or 'recommend' in d['review_summary'].lower() else 0

    positive_words = [
        "comfortable", "perfect", "stylish", "fashionable", "elegant", "cute", "fit",
        "love", "beautiful", "amazing", "flattering", "soft", "chic", "adorable", 
        "gorgeous", "high-quality", "great", "favorite", "versatile", "recommended"
    ]
    negative_words = [
        "uncomfortable", "tight", "loose", "cheap", "poor", "bad", "ill-fitting", 
        "hate", "dislike", "ugly", "stiff", "scratchy", "itchy", "heavy", 
        "unflattering", "low-quality", "awkward", "disappointing", "not fit"
    ]
    positive_count = sum(d['review_text'].lower().count(word) + d['review_summary'].lower().count(word) for word in positive_words)
    negative_count = sum(d['review_text'].lower().count(word) + d['review_summary'].lower().count(word) for word in negative_words)
    return [1, len(d['review_text']), 
            len(d['review_summary']), 
            d['review_text'].count('!') + d['review_summary'].count('!'),
            int(d['review_date'][-4:]),
            age_size_interaction,
            sentiment_count,
            unique_words,
            contains_recommend,
            positive_count,
            negative_count
            ] 
Xtrain = [feat(d) for d in dataTrain]
Xtest = [feat(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.761240801167964

In [35]:
rented_for_categories = ['date', 'party', 'formal affair', 'everyday', 'other', 'wedding', 'vacation', 'party: cocktail', 'work']
def feat(d):
    age = int(d['age'])
    size = int(d['size'])
    age_size_interaction = age * size
    sentiment_words = ['love', 'hate', 'perfect', 'amazing', 'awful', 'good', 'bad']
    sentiment_count = sum(d['review_text'].lower().count(word) + d['review_summary'].lower().count(word) for word in sentiment_words)
    unique_words = len(set(re.findall(r'\w+', d['review_text'] + " " + d['review_summary'])))
    contains_recommend = 1 if 'recommend' in d['review_text'].lower() or 'recommend' in d['review_summary'].lower() else 0
    rented_for_encoded = [1 if d.get('rented for', '').lower() == category else 0 for category in rented_for_categories]

    return [1, len(d['review_text']), 
            len(d['review_summary']), 
            d['review_text'].count('!') + d['review_summary'].count('!'),
            int(d['review_date'][-4:]),
            age_size_interaction,
            sentiment_count,
            unique_words,
            contains_recommend
            ] + rented_for_encoded

Xtrain = [feat(d) for d in dataTrain]
Xtest = [feat(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.819858732405135

In [36]:
rented_for_categories = ['date', 'party', 'formal affair', 'everyday', 'other', 'wedding', 'vacation', 'party: cocktail', 'work']
def feat(d):
    age = int(d['age'])
    size = int(d['size'])
    age_size_interaction = age * size
    sentiment_words = ['love', 'hate', 'perfect', 'amazing', 'awful', 'good', 'bad']
    sentiment_count = sum(d['review_text'].lower().count(word) + d['review_summary'].lower().count(word) for word in sentiment_words)
    unique_words = len(set(re.findall(r'\w+', d['review_text'] + " " + d['review_summary'])))
    contains_recommend = 1 if 'recommend' in d['review_text'].lower() or 'recommend' in d['review_summary'].lower() else 0
    rented_for_encoded = [1 if d.get('rented for', '').lower() == category else 0 for category in rented_for_categories]
    positive_words = [
        "comfortable", "perfect", "stylish", "fashionable", "elegant", "cute", "fit",
        "love", "beautiful", "amazing", "flattering", "soft", "chic", "adorable", 
        "gorgeous", "high-quality", "great", "favorite", "versatile", "recommended"
    ]
    negative_words = [
        "uncomfortable", "tight", "loose", "cheap", "poor", "bad", "ill-fitting", 
        "hate", "dislike", "ugly", "stiff", "scratchy", "itchy", "heavy", 
        "unflattering", "low-quality", "awkward", "disappointing", "not fit"
    ]
    positive_count = sum(d['review_text'].lower().count(word) + d['review_summary'].lower().count(word) for word in positive_words)
    negative_count = sum(d['review_text'].lower().count(word) + d['review_summary'].lower().count(word) for word in negative_words)

    return [1, len(d['review_text']), 
            len(d['review_summary']), 
            d['review_text'].count('!') + d['review_summary'].count('!'),
            int(d['review_date'][-4:]),
            age_size_interaction,
            sentiment_count,
            unique_words,
            contains_recommend,
            positive_count,
            negative_count
            ] + rented_for_encoded

Xtrain = [feat(d) for d in dataTrain]
Xtest = [feat(d) for d in dataTest]
mod = linear_model.LinearRegression(fit_intercept=False)
mod.fit(Xtrain,ytrain)
predictions = mod.predict(Xtest)
mse = MSE(ytest, predictions)
mse

1.7520070352514556

In [37]:
# # Convert to a DataFrame
# df = pd.DataFrame(svd_data, columns=['user_id', 'item_id', 'rating'])

# # Step 2: Load data into surprise's Dataset format
# reader = Reader(rating_scale=(1, 10))  # Assuming ratings are on a scale of 1 to 10
# model_data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)
# trainset, temp_set = train_test_split(model_data, test_size=0.4)

# validation_set, test_set = train_test_split(temp_set, test_size=0.5)

In [38]:
from sklearn.model_selection import train_test_split as sklearn_train_test_split
df = pd.DataFrame(svd_data, columns=['user_id', 'item_id', 'rating'])
train_df, temp_df = sklearn_train_test_split(df, test_size=0.4)
validation_df, test_df = sklearn_train_test_split(temp_df, test_size=0.5)
reader = Reader(rating_scale=(0, 10))
trainset = Dataset.load_from_df(train_df[['user_id', 'item_id', 'rating']], reader).build_full_trainset()
validation_data = [
    (row['user_id'], row['item_id'], row['rating']) for _, row in validation_df.iterrows()
]
test_data = [
    (row['user_id'], row['item_id'], row['rating']) for _, row in test_df.iterrows()
]
param_sets = [
    {'n_factors': 1, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 5, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 10, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 20, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},

    {'n_factors': 1, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 2, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 3, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 4, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 5, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},

    {'n_factors': 1, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 1, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 1, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 1, 'n_epochs': 50, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 1, 'n_epochs': 60, 'lr_all': 0.005, 'reg_all': 0.02},

    {'n_factors': 1, 'n_epochs': 1, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 1, 'n_epochs': 5, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 1, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 1, 'n_epochs': 15, 'lr_all': 0.005, 'reg_all': 0.02},
    {'n_factors': 1, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
]
results = []
for params in param_sets:
    print(f"Testing parameters: {params}")
    model = SVD(**params)
    model.fit(trainset)
    predictions = model.test(validation_data)
    
    true_ratings = [pred.r_ui for pred in predictions]
    predicted_ratings = [pred.est for pred in predictions]
    mse = MSE(true_ratings, predicted_ratings)
    print(f"Validation MSE: {mse}")
    results.append({'params': params, 'mse': mse})

results = sorted(results, key=lambda x: x['mse'])
best_params = results[0]['params']
print("\nBest Parameters on Validation Set:")
print(best_params)

final_model = SVD(**best_params)
final_model.fit(trainset)

test_predictions = final_model.test(test_data)
true_ratings_test = [pred.r_ui for pred in test_predictions]
predicted_ratings_test = [pred.est for pred in test_predictions]
test_mse = MSE(true_ratings_test, predicted_ratings_test)
print(f"Test MSE with Best Parameters: {test_mse}")


Testing parameters: {'n_factors': 1, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
Validation MSE: 1.9383679124388509
Testing parameters: {'n_factors': 5, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
Validation MSE: 1.9395895370302465
Testing parameters: {'n_factors': 10, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
Validation MSE: 1.9418910307830286
Testing parameters: {'n_factors': 20, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
Validation MSE: 1.9438374320605405
Testing parameters: {'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
Validation MSE: 1.9537934896092721
Testing parameters: {'n_factors': 1, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
Validation MSE: 1.9394530281058857
Testing parameters: {'n_factors': 2, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
Validation MSE: 1.9393964578465621
Testing parameters: {'n_factors': 3, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}
Validation MSE: 1.9388948288975265
Testing parameters: 

In [39]:
# model = SVD()
# trainset = model_data.build_full_trainset()
# model.fit(trainset)

In [40]:
# predictions = model.test(testset)

# true_ratings = [pred.r_ui for pred in predictions]
# predicted_ratings = [pred.est for pred in predictions]

# mse = MSE(true_ratings, predicted_ratings)
# mse

In [41]:

# # Define a list of parameter sets to test
# param_sets = [
#     {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
#     {'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
#     {'n_factors': 150, 'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.02},
#     {'n_factors': 200, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.02},
#     {'n_factors': 100, 'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.05},
#     {'n_factors': 200, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},  # Example from your snippet
# ]

# # Initialize a dictionary to store results
# results = []

# # Iterate through each parameter set
# for params in param_sets:
#     print(f"Testing parameters: {params}")
#     # Create and train the model
#     model = SVD(**params)
#     trainset = model_data.build_full_trainset()
#     model.fit(trainset)
    
#     # Make predictions on the test set
#     predictions = model.test(testset)
    
#     # Extract true and predicted ratings
#     true_ratings = [pred.r_ui for pred in predictions]
#     predicted_ratings = [pred.est for pred in predictions]
    
#     # Calculate MSE
#     mse = MSE(true_ratings, predicted_ratings)
#     print(f"MSE: {mse}")
    
#     # Store the results
#     results.append({'params': params, 'mse': mse})

# # Sort results by MSE
# results = sorted(results, key=lambda x: x['mse'])

# # Display the best parameters
# print("\nBest Parameters:")
# print(results[0])


In [42]:
# model = SVD(**{'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02})
# trainset = model_data.build_full_trainset()
# model.fit(trainset)

# predictions = model.test(testset)

# true_ratings = [pred.r_ui for pred in predictions]
# predicted_ratings = [pred.est for pred in predictions]

# mse = MSE(true_ratings, predicted_ratings)
# mse